In [1]:
import statsmodels.formula.api as smf
import pandas as pd

In [2]:
def create_regressor_columns_string(columns):
  # Join all independent variables to define the formula used by the model
  regressor_columns = list(filter(lambda x: x != 'impression_count',columns))
  regressor_columns_string = "+".join(regressor_columns)
  return regressor_columns_string

In [3]:
def regression(path):
    # Load data
    regression_df_pd = pd.read_csv(path)
    regression_df_pd = regression_df_pd.drop('tweet_text', axis=1)
    
    # Define formula without cross terms
    regressor_columns_string = create_regressor_columns_string(regression_df_pd.columns)
    mod = smf.ols(formula=f'impression_count ~ {regressor_columns_string}', data=regression_df_pd)
    res = mod.fit()
    return res

In [ ]:
path_frech_celebrities = '../data/regression/french_celebrities_regression_df.csv'
res_french_celebrities = regression(path_frech_celebrities)

res_french_celebrities.summary()

In [5]:
regressor_columns_string = create_regressor_columns_string(regression_df_pd.columns)
mod = smf.ols(formula=f'impression_count ~ {regressor_columns_string}', data=regression_df_pd)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:       impression_count   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     172.6
Date:                Sat, 13 May 2023   Prob (F-statistic):               0.00
Time:                        20:54:13   Log-Likelihood:            -2.2180e+05
No. Observations:               16151   AIC:                         4.437e+05
Df Residuals:                   16129   BIC:                         4.438e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [ ]:
print(regression_df_pd.columns)

regression_df_pd['impression_count'] = np.log(1+ regression_df_pd['impression_count'])

display(regression_df_pd)

regressor_columns = list(filter(lambda x: x not in [
                         'tweet_external_urls_count', 'dummy_tweet_period_night', 'dummy_joemanchin', 'dummy_chrismurphy', 'dummy_financialservicesbusiness', 'dummy_inflationintheunitedstates', 'dummy_joebiden', 'dummy_politicalfigures', 'dummy_northcarolina', 'dummy_tedcruz', 'dummy_sportsfitnessbusiness', 'dummy_unitedstatescongress', 'dummy_markwarner', 'dummy_politicalnews', 'dummy_chriscoons'], regression_df_pd.columns))

regressor_columns_string = create_regressor_columns_string(
    regressor_columns)
mod = smf.ols(
    formula=f'impression_count ~ {regressor_columns_string}', data=regression_df_pd)
res = mod.fit()
print(res.summary())